In [2]:
import pandas as pd
from pandas import Series, DataFrame
import tushare as ts

1. 获取数据（本地csv）
2. 分别计算ma5，ma30
3. 取【收盘价】分别获取以 ma5 > ma30 与 ma5 < ma30 为判断基准，以trade_date 为行索引的布尔数列
4. 比对 ma5 > ma30 与 ma5 < ma30 数组，计算金叉、死叉日期
5. 针对历史金叉、死叉日期模拟买卖，计算盈亏

In [12]:
df = pd.read_csv("../../../daily/000564.SZ.csv")
df.drop(labels="Unnamed: 0", axis='columns', inplace=True)
df['trade_date'] = pd.to_datetime(df['trade_date'], format='%Y%m%d')
df.set_index('trade_date', inplace=True)
df = df[::-1]
df

,ts_code,open,high,low,close,pre_close,change,pct_chg,vol,amount
trade_date,,,,,,,,,,
1999-08-25,000564.SZ,11.00,11.58,10.82,11.09,10.99,0.10,0.9100,86552.00,97373.7202
1999-08-26,000564.SZ,11.00,11.25,10.75,10.95,11.09,-0.14,-1.2600,30760.00,33674.8802
1999-08-27,000564.SZ,11.20,11.60,11.00,11.21,10.95,0.26,2.3700,73587.00,83685.6528
1999-08-30,000564.SZ,11.50,11.53,10.98,11.02,11.21,-0.19,-1.6900,31796.00,35530.7037
1999-08-31,000564.SZ,11.07,11.15,10.65,10.94,11.02,-0.08,-0.7300,28496.00,30857.2490
...,...,...,...,...,...,...,...,...,...,...
2022-08-01,000564.SZ,1.02,1.02,0.99,1.00,1.03,-0.03,-2.9126,1632772.53,164055.0980
2022-08-02,000564.SZ,1.00,1.00,0.98,0.99,1.00,-0.01,-1.0000,1597561.27,157824.4440
2022-08-03,000564.SZ,0.98,1.00,0.98,0.99,0.99,0.00,0.0000,811593.34,80347.4680


In [16]:
ma5 = df['close'].rolling(5).mean()
ma30 = df['close'].rolling(30).mean()
s1 = ma5 > ma30
s2 = ma5 < ma30

In [17]:
golden_ex = s1 & s2.shift(1)
death_ex = s1.shift(1) & s2
df = df[30:]

trade_date
1999-08-25    False
1999-08-26    False
1999-08-27    False
1999-08-30    False
1999-08-31    False
              ...  
2022-08-01    False
2022-08-02    False
2022-08-03    False
2022-08-04    False
2022-08-05    False
Name: close, Length: 5000, dtype: bool

In [31]:
golden_date = df[golden_ex].index
death_date = df[death_ex].index
golden_series = Series(data=1, index=golden_date)
death_series = Series(data=0, index=death_date)
s = pd.concat(objs=(golden_series, death_series))
s.sort_index(inplace=True)

In [47]:
money_init = 5000
money_result = money_init
hold = 0

for i in range(0, len(s)):
  date = s.index[i]
  if s[i] == 1:
    price = df.loc[s.index[i]]['open']
    hand_buy = money_result // (price * 100)
    if hand_buy > 0:
      hold += hand_buy * 100
      cost = hand_buy * 100 * price
      money_result -= cost
      print(date.strftime(format='%Y-%m-%d'), ' 买入', hold, '股, 价格: ', round(price, 2), ', 花费: ', round(cost,2), ', 剩余资金: ', round(money_result,2))
  else:
    if hold > 0:
      price = df.loc[s.index[i]]['open']
      sell_total = hold
      sell_amount = price * sell_total
      money_result += sell_amount
      hold = 0
      print(date.strftime(format='%Y-%m-%d'), ' 卖出', sell_total, ', 价格: ', round(price, 2), ', 收入: ', round(sell_amount,2), ', 盈亏: ', round(money_result,2))

1999-11-22  买入 500.0 股, 价格:  8.9 , 花费:  4450.0 , 剩余资金:  550.0
1999-12-15  卖出 500.0 , 价格:  8.7 , 收入:  4350.0 , 盈亏:  4900.0
2000-01-27  买入 600.0 股, 价格:  7.91 , 花费:  4746.0 , 剩余资金:  154.0
2000-03-15  卖出 600.0 , 价格:  8.98 , 收入:  5388.0 , 盈亏:  5542.0
2000-03-31  买入 500.0 股, 价格:  9.38 , 花费:  4690.0 , 剩余资金:  852.0
2000-04-03  卖出 500.0 , 价格:  9.0 , 收入:  4500.0 , 盈亏:  5352.0
2000-04-10  买入 500.0 股, 价格:  10.07 , 花费:  5035.0 , 剩余资金:  317.0
2000-06-22  卖出 500.0 , 价格:  10.94 , 收入:  5470.0 , 盈亏:  5787.0
2000-08-01  买入 500.0 股, 价格:  10.24 , 花费:  5120.0 , 剩余资金:  667.0
2000-09-05  卖出 500.0 , 价格:  10.55 , 收入:  5275.0 , 盈亏:  5942.0
2000-10-26  买入 600.0 股, 价格:  9.8 , 花费:  5880.0 , 剩余资金:  62.0
2000-12-13  卖出 600.0 , 价格:  10.08 , 收入:  6048.0 , 盈亏:  6110.0
2001-01-15  买入 500.0 股, 价格:  10.35 , 花费:  5175.0 , 剩余资金:  935.0
2001-04-27  卖出 500.0 , 价格:  11.82 , 收入:  5910.0 , 盈亏:  6845.0
2001-06-15  买入 500.0 股, 价格:  11.82 , 花费:  5910.0 , 剩余资金:  935.0
2001-07-05  卖出 500.0 , 价格:  11.8 , 收入:  5900.0 , 盈亏:  6835.0
2001-